# Comprehensive Research: Customer Segmentation (RFM)

## 1. Environment & Data
**Objective**: Find hidden patterns in customer behavior.
**Technique**: K-Means Clustering on RFM (Recency, Frequency, Monetary) vectors.
**Research Questions**:
1.  How many clusters are optimal? (Elbow vs Silhouette)
2.  Do the clusters align with business intuition (e.g., "VIPs", "Churned")?


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

sns.set(style="whitegrid")

# --- 1. DATA GENERATION ---
def make_rfm_data(n=1000):
    # Cluster 1: VIPs (Recent, Frequent, Rich)
    c1_r = np.random.normal(5, 2, 100)
    c1_f = np.random.normal(50, 10, 100)
    c1_m = c1_f * 100
    
    # Cluster 2: New Users (Recent, Infrequent, Low spend)
    c2_r = np.random.normal(10, 5, 400)
    c2_f = np.random.normal(2, 1, 400)
    c2_m = c2_f * 20
    
    # Cluster 3: Churned (Old, Infrequent)
    c3_r = np.random.normal(100, 20, 500)
    c3_f = np.random.normal(5, 2, 500)
    c3_m = c3_f * 50
    
    df = pd.DataFrame({
        'Recency': np.concatenate([c1_r, c2_r, c3_r]),
        'Frequency': np.concatenate([c1_f, c2_f, c3_f]),
        'Monetary': np.concatenate([c1_m, c2_m, c3_m])
    })
    # Fix absolute values
    df = df.abs()
    return df

df = make_rfm_data()
print("Data Generated.")

## 2. EDA: Skewness Check
RFM data, especially Monetary, follows Power Laws (Pareto Principle). Standard K-Means fails on skewed data.

In [ ]:
sns.pairplot(df)
plt.show()

**Observation**: The data is bunched near zero with long tails. **Log Transformation** is mandatory.

In [ ]:
# Log Transform + Scale
df_log = np.log1p(df)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_log)

print("Data Transformed.")

## 3. Finding Optimal K (Hyperparameter Tuning)
We use two metrics:
1.  **Inertia (Elbow Method)**: Sum of distances to centroid.
2.  **Silhouette Score**: Separation distance between clusters.

In [ ]:
inertia = []
silhouette = []
K_range = range(2, 10)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)
    silhouette.append(silhouette_score(X_scaled, kmeans.labels_))

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(K_range, inertia, 'bo-')
plt.title("Elbow Method (Inertia)")
plt.xlabel("K")

plt.subplot(1, 2, 2)
plt.plot(K_range, silhouette, 'ro-')
plt.title("Silhouette Score (Higher is Better)")
plt.xlabel("K")
plt.show()

**Observation**: Elbow suggests 3. Silhouette suggests 3. We proceed with K=3.

In [ ]:
model = KMeans(n_clusters=3, random_state=42, n_init=10)
labels = model.fit_predict(X_scaled)
df['Cluster'] = labels

## 4. Cluster Profiling (Snake Plot)
To understand the clusters, we plot their Standardized Mean Values.

In [ ]:
df_melt = pd.DataFrame(X_scaled, columns=['Recency', 'Frequency', 'Monetary'])
df_melt['Cluster'] = labels
df_melt = pd.melt(df_melt, id_vars=['Cluster'], var_name='Metric', value_name='Value')

plt.figure(figsize=(10, 6))
sns.lineplot(data=df_melt, x='Metric', y='Value', hue='Cluster', palette='bright')
plt.title("Snake Plot (Cluster Profiles)")
plt.show()

**Analysis**:
*   **Cluster 0**: Low Recency, High Frequency/Monetary -> **VIPs**.
*   **Cluster 1**: High Recency, Low Frequency -> **Churned**.
*   **Cluster 2**: Mid Recency, Low Frequency -> **New Users**.